In [5]:
# Azure Imports
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    CorsOptions,
    SearchIndex,
    SearchFieldDataType,
    SimpleField,
    SearchableField
)

import json
import time

In [2]:
# Boilerplate code to connect to Azure Search Service
service_name = "college-search-service"
admin_key = "B6C37FA89C0A5BDD60267B82A1DC1BEF"

index_name = "college-search-index"

# Create an SDK client
endpoint = "https://{}.search.windows.net/".format(service_name)
admin_client = SearchIndexClient(endpoint=endpoint,
                    index_name=index_name,
                    credential=AzureKeyCredential(admin_key))

search_client = SearchClient(endpoint=endpoint,
                    index_name=index_name,
                    credential=AzureKeyCredential(admin_key))

In [6]:
# basic config
LINKS_PATH = "./data/college_links.json"

MAP_PATH = "./data/college_map.json"

college_map = {}

In [39]:
# map college names of collegedata website and CST dataset

with open(LINKS_PATH) as f:
    colleges = json.load(f)
    for college in colleges["colleges"]:
        filter = ""
        result_name = ""
        # search the most related college name based on collegedata website
        results = search_client.search(
            search_text=college["college_name"],
            top=10,
            search_mode="all",
            search_fields=["INSTNM"],
            filter=filter
        )

        for result in results:
            if result is not None:
                # if the names match exactly
                if result["INSTNM"] == college["college_name"]:
                    result_name = result["INSTNM"]
                # otherwise use city and state abbreviation to help get the most related name
                else:
                    if college["state"] is not "DC":
                        filter = "CITY eq \'{}\' and STABBR eq \'{}\'".format(college["city"], college["state"])
                    else:
                        filter = "STABBR eq 'DC'"
                    results_sec = search_client.search(
                        search_text=college["college_name"],
                        top=1,
                        search_mode="all",
                        search_fields=["INSTNM"],
                        filter=filter
                    )

                    for result_sec in results_sec:
                        if result_sec is not None:
                            result_name = result_sec["INSTNM"]
                        break
            break

        college_map[college["college_name"]] = result_name

In [40]:
with open(MAP_PATH, "w") as f:
    f.write(json.dumps(college_map, indent=4))

## Single College Testing

In [41]:
# test
results = search_client.search(
  search_text="State University of New York at Oswego",
  top=5,
  search_mode="all",
  search_fields=["INSTNM"]
)

In [42]:
for result_element in results:
    print(result_element["INSTNM"])